In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using Plots
using Test
include("car.jl");
using ForwardDiff

  Activating environment at `c:\Users\ADMIN\Documents\GitHub\ocrl_project\Project.toml`
   Installed TriangularSolve ────────────────── v0.1.11
   Installed FillArrays ─────────────────────── v0.13.1
   Installed CloseOpenIntervals ─────────────── v0.1.6
   Installed CPUSummary ─────────────────────── v0.1.15
   Installed VectorizationBase ──────────────── v0.21.25
   Installed SciMLBase ──────────────────────── v1.28.0
   Installed ControlSystems ─────────────────── v0.12.11
   Installed LoopVectorization ──────────────── v0.12.103
   Installed PDMats ─────────────────────────── v0.11.7
   Installed DelayDiffEq ────────────────────── v5.35.1
   Installed Polyester ──────────────────────── v0.6.7
   Installed SentinelArrays ─────────────────── v1.3.12
   Installed DualNumbers ────────────────────── v0.6.7
   Installed DiffEqCallbacks ────────────────── v2.22.0
   Installed Quaternions ────────────────────── v0.5.3
   Installed OSQP ───────────────────────────── v0.8.0
   Installed Expo

In [ ]:
# Define the model
model = BicycleModel()

# get the number of states and controls
n = state_dim(model)
m = control_dim(model)

# Evaluate the continuous and discrete dynamics
x0 = SA[0.,0.,0.,0.]
u0 = SA[0.,0.]
t0 = 0.0
dt = 0.025
dynamics(model, x0, u0)
discrete_dynamics(RK4, model, x0, u0, t0, dt)  # use rk4 for integration

# Evaluate the continuous and discrete Jacobians
z0 = KnotPoint(x0,u0,dt,t0)   # create a `KnotPoint` type that stores everything together
∇f = RobotDynamics.DynamicsJacobian(model)
jacobian!(∇f, model, z0)
discrete_jacobian!(RK4, ∇f, model, z0)

# Extract pieces of the Jacobian
A = ∇f.A
B = ∇f.B;


LoadError: UndefVarError: BicycleModel not defined

In [ ]:
nx = n
nu = m

Q = Diagonal([1,1,1e-2,1e-2])
R = Diagonal([1e-1,1e-1])
Qf = Diagonal([1,1,1,1.])*10

N = 101 #401
Nmpc = 10 #60
Nhz = 1
delta = 12 #80
Nc = N + 2*delta

FD = ForwardDiff;

LoadError: UndefVarError: n not defined

In [ ]:
Q = Diagonal([10,10,10,1.])
R = Diagonal([1.,1.])
Qf = Diagonal([1,1,1,1.])*1;

LoadError: UndefVarError: Diagonal not defined

In [ ]:
function stage_cost_convoy(x, x_ref, Q)
    # LQR cost at each knot point (depends on both x and u)
    J = 0.5 * (x - xref)' * Q * (x - xref)
    
    return J
end

function trajectory_cost_convoy(X, Xref, Q, Nmpc)
    # calculate the cost of a given trajectory 
    J = 0.0
    for i = 1:Nmpc
        J += stage_cost_convoy(X[i], Xref[i], Q)
    end
    
    return J
end
        
function stage_cost_expansion_convoy(x, xref, Q, Nmpc)
    # if the stage cost function is J, return the following derivatives:
    # ∇²ₓJ,  ∇ₓJ, ∇²ᵤJ, ∇ᵤJ
    Jxx = Q
    Jx = Q * (x - xref)
    
    return Jxx, Jx
end

function total_cost(X,U,Xfoll,Xlead, K,d,ΔJ, Q, R, Qf, Qfoll, Qlead, Nmpc)
    #J_traj = trajectory_cost(X,U,Xref,Uref, Q, R, Qf, Nmpc)
    J_foll = trajectory_cost_convoy(X, Xfoll, Qfoll, Nmpc)
    J_lead = trajectory_cost_convoy(X, Xlead, Qlead, Nmpc)
    return J_traj + J_foll + J_lead
end

total_cost (generic function with 1 method)

In [ ]:
function dynamics_jacobians(x,u,dt)
    # returns the discrete time dynamics jacobians
    A = FD.jacobian(_x -> discrete_dynamics(RK4, model, _x, u, t0, dt),x)
    B = FD.jacobian(_u -> discrete_dynamics(RK4, model, x, _u, t0, dt),u)
    return A,B
end